<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bbae8397c672d7dc544f0610f30ac72d94d16cf08fdb9d43d88829305aa46622
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2


    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-31 14:53:28
-------------------
qualified stocks: 88
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.37 K
Current:  1.43 C
-------------------
Today PnL: 1.24 L (0.87%)
Current PnL: -25.94 L (-16.54%)
CY Booked + Current PnL: -11.65 L (-7.43%)
-------------------
Total profit:  1.57 L
Total loss:  -27.50 L
-------------------
Total Booked + Current PnL: 15.33 L (11.81%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.99%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 91.31 L (63.81%)
Deployed:  1.30 C
Current:  1.43 C
CAGR/XIRR %: 5.39%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.00,0.77,-19.78,24.77,0.09,20026.0,-19930.0,80847.0,87.62,39.0,M-SC,3.18,253.0,-1.00,0.56,6.21,OX40N,NTT,DURABLES
53,NESTLEIND,1377.00,1.19,13.66,6.80,21.40,20515.0,36270.0,301696.0,5.44,67.0,X-LC,5.20,12.0,1.77,2.11,20.86,XY25,NTT,FMCG
51,MASFIN,398.61,2.83,-1.00,23.28,22.05,22583.0,-975.0,97005.0,-14.63,57.0,H-SC,6.61,164.0,-0.04,0.68,41.00,XR,ATH,FINANCE
84,VOLTAS,1530.00,0.93,6.64,12.24,19.69,25028.0,12738.0,204480.0,-3.70,47.0,X-MC,2.13,78.0,0.51,1.43,13.89,XY25,NTT,AC
44,ITC,452.00,0.70,-1.13,12.05,10.78,28631.0,-2715.0,237603.0,-41.15,50.0,X-LC,1.99,5.0,-0.09,1.66,4.05,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,5.40,0.52,46.19,46.95,58176.0,653.0,125950.0,-7.26,69.0,H-SC,9.39,154.0,0.01,0.88,57.79,MH,ATH,POWER
66,SHALBY,327.00,3.35,-22.30,61.00,25.10,85861.0,-40388.0,140755.0,935.09,50.0,M-SC,9.13,250.0,-0.47,0.98,12.80,XY24,NTT,HEALTHCARE
10,BAJAJHLDNG,14451.52,3.23,2.00,27.47,30.03,37371.0,2674.0,136044.0,-1.33,51.0,X-LC,12.16,35.0,0.07,0.95,10.68,X40,ATH,FINANCE
52,MEDANTA,1486.00,3.04,-0.95,25.26,24.08,31764.0,-1202.0,125748.0,-9.02,50.0,X-SC,7.59,89.0,-0.04,0.88,18.48,XY24,NTT,HEALTHCARE
23,DIXON,18931.72,2.94,-19.34,56.29,26.07,88639.0,-37752.0,157469.0,-60.75,31.0,X-MC,16.08,56.0,-0.43,1.10,2.94,X40N,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJESHEXPO,518.0,-4.19,-64.45,181.52,0.07,88509.0,-88417.0,48760.0,1686.62,42.0,L-SC,4.35,268.0,-1.00,0.34,21.27,OX40N,NTT,JEWELLERY
28,GLAXO,3466.2,-3.76,-4.29,45.38,39.15,86556.0,-8548.0,190736.0,-23.10,28.0,X-MC,10.68,60.0,-0.10,1.33,23.47,X40,ATH,PHARMA
56,QUESS,424.0,-2.29,-30.78,106.36,42.84,47860.0,-20008.0,44998.0,-54.82,40.0,X-SC,31.64,83.0,-0.42,0.31,1.80,XY24,NTT,MISC
19,COFFEEDAY,80.0,-1.33,-37.49,116.39,35.27,82616.0,-42567.0,70982.0,-51.71,55.0,L-SC,7.00,270.0,-0.52,0.50,73.73,XR,NTT,HOTELS
8,AWL,485.0,-1.25,-24.99,104.64,53.50,236837.0,-75416.0,226335.0,-63.63,34.0,X-MC,6.98,58.0,-0.32,1.58,0.53,XY24,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.84,-0.24,82.11,81.67,76843.0,-224.0,93586.0,8427.78,67.0,L-SC,16.05,271.0,-0.00,0.65,81.23,XR,NTT,CERAMICS
49,KPIGREEN,730.66,5.40,0.52,46.19,46.95,58176.0,653.0,125950.0,-7.26,69.0,H-SC,9.39,154.0,0.01,0.88,57.79,MH,ATH,POWER


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,2.45,-5.05,118.96,107.91,166775.0,-7450.0,140194.0,-22.58,46.0,M-SC,11.59,216.0,-0.04,0.98,4.53,OX40N,ATH,CABLES
40,INDIGOPNTS,1408.00,2.09,-18.59,22.86,0.02,32485.0,-32455.0,142104.0,116.52,40.0,M-SC,8.38,234.0,-1.00,0.99,23.61,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,0.77,-19.78,24.77,0.09,20026.0,-19930.0,80847.0,87.62,39.0,M-SC,3.18,253.0,-1.00,0.56,6.21,OX40N,NTT,DURABLES
48,KANSAINER,340.00,0.87,-21.27,44.13,13.47,93692.0,-57357.0,212310.0,-67.73,55.0,H-SC,6.49,158.0,-0.61,1.48,7.95,XY24,NTT,PAINTS
68,SIS,528.00,0.00,-23.38,58.27,21.26,49569.0,-25964.0,85068.0,2007.77,52.0,H-SC,4.48,166.0,-0.52,0.59,14.94,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,5.40,0.52,46.19,46.95,58176.0,653.0,125950.0,-7.26,69.0,H-SC,9.39,154.0,0.01,0.88,57.79,MH,ATH,POWER
86,WIPRO,420.00,-0.19,7.93,59.60,72.27,109002.0,13441.0,182889.0,-7.44,56.0,M-LC,5.23,99.0,0.12,1.28,14.60,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.00,-0.19,7.93,59.60,72.27,109002.0,13441.0,182889.0,-7.44,56.0,M-LC,5.23,99.0,0.12,1.28,14.60,XR,NTT,IT
49,KPIGREEN,730.66,5.40,0.52,46.19,46.95,58176.0,653.0,125950.0,-7.26,69.0,H-SC,9.39,154.0,0.01,0.88,57.79,MH,ATH,POWER
6,ASIANTILES,137.00,0.84,-0.24,82.11,81.67,76843.0,-224.0,93586.0,8427.78,67.0,L-SC,16.05,271.0,-0.00,0.65,81.23,XR,NTT,CERAMICS
51,MASFIN,398.61,2.83,-1.00,23.28,22.05,22583.0,-975.0,97005.0,-14.63,57.0,H-SC,6.61,164.0,-0.04,0.68,41.00,XR,ATH,FINANCE
26,FINCABLES,1641.55,2.45,-5.05,118.96,107.91,166775.0,-7450.0,140194.0,-22.58,46.0,M-SC,11.59,216.0,-0.04,0.98,4.53,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.02,-19.32,92.70,55.47,181963.0,-46994.0,196292.0,-28.46,26.0,X-MC,16.46,64.0,-0.26,1.37,0.58,X40N,ATH,FINANCE
28,GLAXO,3466.20,-3.76,-4.29,45.38,39.15,86556.0,-8548.0,190736.0,-23.10,28.0,X-MC,10.68,60.0,-0.10,1.33,23.47,X40,ATH,PHARMA
35,HONAUT,58357.33,-0.09,-19.04,77.03,43.32,101572.0,-31012.0,131860.0,-30.40,28.0,X-SC,7.22,90.0,-0.31,0.92,1.24,X40N,ATH,ELECTRICAL
23,DIXON,18931.72,2.94,-19.34,56.29,26.07,88639.0,-37752.0,157469.0,-60.75,31.0,X-MC,16.08,56.0,-0.43,1.10,2.94,X40N,ATH,IT
8,AWL,485.00,-1.25,-24.99,104.64,53.50,236837.0,-75416.0,226335.0,-63.63,34.0,X-MC,6.98,58.0,-0.32,1.58,0.53,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.5,1.13,-17.68,52.47,25.51,80379.0,-32905.0,153190.0,-6.04,41.0,H-LC,0.98,49.0,-0.41,1.07,12.89,AR,ATH,ELECTRICAL
80,UNITDSPR,1644.0,1.38,6.03,13.90,20.77,34509.0,14121.0,248265.0,-3.23,55.0,X-MC,1.59,62.0,0.41,1.73,13.37,X40N,NTT,BREWERIES
13,BERGEPAINT,680.0,2.00,-4.50,26.85,21.14,58291.0,-10239.0,217100.0,-11.18,44.0,X-MC,1.62,74.0,-0.18,1.52,20.56,XY24,NTT,PAINTS
21,DABUR,735.0,1.75,-1.40,45.75,43.70,111898.0,-3482.0,244586.0,-9.61,56.0,X-MC,1.98,72.0,-0.03,1.71,13.44,XY24,BTT,FMCG
44,ITC,452.0,0.70,-1.13,12.05,10.78,28631.0,-2715.0,237603.0,-41.15,50.0,X-LC,1.99,5.0,-0.09,1.66,4.05,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,0.46,-39.21,122.42,35.20,95751.0,-50455.0,78215.0,2.53,35.0,X-SC,17.37,92.0,-0.53,0.55,0.46,X40,NTT,FOOTWEAR
8,AWL,485.00,-1.25,-24.99,104.64,53.50,236837.0,-75416.0,226335.0,-63.63,34.0,X-MC,6.98,58.0,-0.32,1.58,0.53,XY24,NTT,FMCG
9,BAJAJHFL,181.50,0.02,-19.32,92.70,55.47,181963.0,-46994.0,196292.0,-28.46,26.0,X-MC,16.46,64.0,-0.26,1.37,0.58,X40N,ATH,FINANCE
3,ACC,3906.00,0.40,-26.92,124.61,64.15,216697.0,-64051.0,173900.0,-57.19,36.0,X-SC,5.65,82.0,-0.30,1.22,0.80,XY24,BTT,CEMENT
20,COLPAL,3726.84,1.00,-21.25,79.69,41.51,165277.0,-55965.0,207400.0,-6.31,37.0,X-MC,8.38,61.0,-0.34,1.45,1.00,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-2.29,-30.78,106.36,42.84,47860.0,-20008.0,44998.0,-54.82,40.0,X-SC,31.64,83.0,-0.42,0.31,1.80,XY24,NTT,MISC
60,RELAXO,1176.00,1.01,-47.62,190.77,52.31,145056.0,-69123.0,76037.0,-44.25,46.0,X-SC,7.38,91.0,-0.48,0.53,1.78,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,0.46,-39.21,122.42,35.20,95751.0,-50455.0,78215.0,2.53,35.0,X-SC,17.37,92.0,-0.53,0.55,0.46,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,3.04,-0.95,25.26,24.08,31764.0,-1202.0,125748.0,-9.02,50.0,X-SC,7.59,89.0,-0.04,0.88,18.48,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,-0.09,-19.04,77.03,43.32,101572.0,-31012.0,131860.0,-30.40,28.0,X-SC,7.22,90.0,-0.31,0.92,1.24,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-0.87,-11.51,36.40,20.71,121836.0,-43523.0,334714.0,-23.62,49.0,X-LC,3.03,1.0,-0.36,2.34,11.84,X40,ATH,IT
42,INFY,1972.00,-0.04,10.34,21.66,34.24,74782.0,32364.0,345252.0,-14.21,54.0,X-LC,7.33,2.0,0.43,2.41,19.48,X40,NTT,IT
77,TMPV,600.00,1.29,-24.90,63.76,22.98,108164.0,-56255.0,169643.0,-21.86,58.0,X-LC,5.84,3.0,-0.52,1.19,6.70,XY24,NTT,AUTO
83,VBL,671.64,1.04,-1.12,37.21,35.67,116207.0,-3541.0,312301.0,-12.80,62.0,X-LC,2.91,4.0,-0.03,2.18,12.58,X40N,ATH,FMCG
44,ITC,452.00,0.70,-1.13,12.05,10.78,28631.0,-2715.0,237603.0,-41.15,50.0,X-LC,1.99,5.0,-0.09,1.66,4.05,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,0.23,-37.87,113.90,32.89,54633.0,-29240.0,47966.0,-692.31,50.0,L-MC,6.71,259.0,-0.54,0.34,32.09,XR,NTT,BANKS
6,ASIANTILES,137.00,0.84,-0.24,82.11,81.67,76843.0,-224.0,93586.0,8427.78,67.0,L-SC,16.05,271.0,-0.00,0.65,81.23,XR,NTT,CERAMICS
51,MASFIN,398.61,2.83,-1.00,23.28,22.05,22583.0,-975.0,97005.0,-14.63,57.0,H-SC,6.61,164.0,-0.04,0.68,41.00,XR,ATH,FINANCE
14,BSOFT,831.70,2.47,-21.31,91.61,50.79,101397.0,-29970.0,110683.0,0.89,55.0,H-SC,11.16,151.0,-0.30,0.77,28.87,XR,ATH,IT
49,KPIGREEN,730.66,5.40,0.52,46.19,46.95,58176.0,653.0,125950.0,-7.26,69.0,H-SC,9.39,154.0,0.01,0.88,57.79,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.84,-0.24,82.11,81.67,76843.0,-224.0,93586.0,8427.78,67.0,L-SC,16.05,271.0,-0.00,0.65,81.23,XR,NTT,CERAMICS
49,KPIGREEN,730.66,5.40,0.52,46.19,46.95,58176.0,653.0,125950.0,-7.26,69.0,H-SC,9.39,154.0,0.01,0.88,57.79,MH,ATH,POWER
14,BSOFT,831.70,2.47,-21.31,91.61,50.79,101397.0,-29970.0,110683.0,0.89,55.0,H-SC,11.16,151.0,-0.30,0.77,28.87,XR,ATH,IT
76,TITAGARH,1548.00,1.67,-15.91,73.32,45.74,149307.0,-38539.0,203638.0,12.11,64.0,H-SC,4.86,174.0,-0.26,1.42,32.45,XY24,NTT,ENGINEERING
37,ICICIPRULI,790.00,2.03,10.56,18.24,30.73,39730.0,20812.0,217817.0,-15.30,70.0,X-MC,4.30,75.0,0.52,1.52,24.68,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.96
1,25,44.12
2,50,75.08


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.24
MC    30.11
LC    25.64
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.47
X40      23.47
X40N     12.63
XY25      9.95
XR        9.65
AR        8.62
OX40N     7.34
SR        0.98
MH        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.70
H-SC    23.60
X-LC    22.26
M-SC    11.28
X-SC     7.87
H-MC     4.70
L-SC     1.49
M-MC     1.37
M-LC     1.28
H-LC     1.07
L-LC     1.03
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.10,-6.64,40.93
IT,12.54,-17.24,76.19
FINANCE,10.45,-15.41,64.40
MISC,6.99,-29.68,82.72
ELECTRICAL,5.81,-12.74,54.04
PAINTS,5.70,-13.20,30.01
INSURANCE,4.70,-0.81,35.31
PHARMA,3.95,-3.78,36.45
AUTO,2.83,-29.99,75.06


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3133391.0,21
AR,1338257.0,10
XR,1282351.0,13
X40,1037335.0,15
X40N,876298.0,9
OX40N,741333.0,10
XY25,384090.0,7
SR,280258.0,2
MH,58176.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3602290.0,25
M-SC,1443197.0,15
X-MC,1432959.0,16
X-LC,882791.0,13
X-SC,792607.0,8
H-MC,405234.0,3
L-SC,247968.0,3
M-LC,109002.0,1
H-LC,80379.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1236671.0      6
           AR         915936.0      5
M-SC       XY24       815341.0      6
H-SC       XR         781226.0      7
X-MC       X40        487248.0      7
           XY24       407026.0      3
X-LC       X40        369048.0      6
X-MC       X40N       348380.0      4
H-SC       OX40N      330023.0      4
M-SC       OX40N      322801.0      5
X-SC       X40N       315247.0      3
           XY24       296321.0      3
H-SC       SR         280258.0      2
H-MC       AR         214918.0      2
X-LC       X40N       212671.0      2
H-MC       XY24       190316.0      1
X-MC       XY25       190305.0      2
X-LC       XY24       187716.0      2
X-SC       X40        181039.0      2
M-SC       XR         178031.0      2
L-SC       XR         159459.0      2
M-SC       AR         127024.0      2
X-LC       XY25       113356.0      3
M-LC       XR         109002.0      1
L-SC       OX40N       88509.0      1
H-LC       AR          80379.0      1
H-SC       MH          58176.0      1
L-MC       XR          54633.0      1
M-MC       XY25        48808.0      1
L-LC       XY25        31621.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
